In [ ]:
!pip install -U efficientnet

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import matplotlib.pyplot as plt
import json
from tensorflow.keras.utils import Sequence,to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import *
import efficientnet.keras as efn 
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
import tensorflow.compat.v1 as tc
import tensorflow as tf
import albumentations as A
from tqdm import tqdm
from sklearn.utils import shuffle
from tensorflow.keras.optimizers import *

In [ ]:
train_df=pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/train.csv')
sample_sub=pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/sample_submission.csv')
train_df.head(10)

In [ ]:
train_df.shape

In [ ]:
with open('/kaggle/input/cassava-leaf-disease-classification/label_num_to_disease_map.json') as f:
    label_dic=json.load(f)
print(label_dic)

In [ ]:
train_img_path='/kaggle/input/cassava-leaf-disease-classification/train_images/'
test_img_path='/kaggle/input/cassava-leaf-disease-classification/test_images/'
weight_path='/kaggle/input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
fig=plt.figure(figsize=(15,15))
r,c=3,3
for i in range(1,r*c+1):
    img=cv2.imread(os.path.join(train_img_path,train_df.loc[i,'image_id']))
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    t=train_df.loc[i,'label']
    fig.add_subplot(r,c,i)
    plt.imshow(img)
    plt.title(label_dic[str(t)])

In [ ]:
BATCH_SIZE=15
DIMS=(224,224,3)

In [ ]:
train_df_,val_df=train_test_split(train_df,test_size=0.2)
train_ids,train_lbls=train_df_['image_id'].values,train_df_['label'].values
val_ids,val_lbls=val_df['image_id'].values,val_df['label'].values
print('Number of samples in train and val: {},{} '.format(len(train_ids),len(val_ids)))

In [ ]:
'''
#Augmentation
transform = A.Compose([
            A.HorizontalFlip(),
            A.VerticalFlip(),
            A.CenterCrop(224,224),
            A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.50, rotate_limit=45, p=.75),
            A.GridDistortion(),])

#augmented_image = transform(image=image)['image']

fig=plt.figure(figsize=(15,15))
r,c=4,2
for i in range(1,r*c+1):
    img=cv2.imread(os.path.join(train_img_path,train_df.loc[i,'image_id']))
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    if i%2==0:
        img = transform(image=img)['image']
    fig.add_subplot(r,c,i)
    plt.imshow(img)
'''

In [ ]:
class customGen(Sequence):
    def __init__(self,data,path,batch_size=BATCH_SIZE,dims=DIMS,shuffle_=True,is_train=True):
        self.img_ids=data[0]
        self.labels=data[1]
        self.batch_size=batch_size
        self.dims=DIMS
        self.is_train=is_train
        self.on_epoch_end()
        self.path=path
        
    
    def __len__(self):
        return int(len(self.img_ids)//self.batch_size)
    
    def on_epoch_end(self):
        self.indexes=np.arange(len(self.img_ids))        
        if self.is_train:
                np.random.shuffle(self.indexes)
            
    def __getitem__(self,index):
        indexes=self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        img_ix=[self.img_ids[i] for i in indexes]
        y=[self.labels[i] for i in indexes]
        X=self.__data_generation(img_ix)
        return X,to_categorical(y,num_classes=5)
    
    def overlap_image(self,x):
        sec_img=np.random.choice(self.img_ids)
        img=cv2.imread(os.path.join(self.path,sec_img))
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img=cv2.resize(img,(self.dims[0],self.dims[1]))
        overlay=cv2.addWeighted(x,0.8,img,0.25,0)
        return overlay
        
    
    def augment_(self,image):
        transform = A.Compose([
            A.HorizontalFlip(),
            A.VerticalFlip(),
            A.CenterCrop(224,224),
            A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.50, rotate_limit=45, p=.75),
            A.GridDistortion(),])
        
        augmented_image = transform(image=image)['image']
        return augmented_image
    
    
    def __data_generation(self,ix):
        tmp=np.zeros((self.batch_size,*self.dims))
        for ix_,img_id in enumerate(ix):
            img=cv2.imread(os.path.join(self.path,img_id))
            img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img=cv2.resize(img,(self.dims[0],self.dims[1]))            
            if self.is_train:
                #Dont always apply augmentation
                if np.random.randint(2)==1:
                    img=self.augment_(img)
                #img=self.overlap_image(img)
            tmp[ix_]=img.astype('float')/255.
        return tmp
            

In [ ]:
train_gen=customGen([train_ids,train_lbls],train_img_path)
val_gen=customGen([val_ids,val_lbls],train_img_path,is_train=False)

In [ ]:
input_=Input(DIMS)
x0=Conv2D(3,5,strides=1,padding='same')(input_)
x0=BatchNormalization()(x0)
x0=Activation('relu')(x0)
base_feat0=efn.EfficientNetB0(weights='imagenet',include_top=False)(x0)
x0=Reshape((base_feat0.shape[1]*base_feat0.shape[2],base_feat0.shape[3]))(base_feat0)

x1=Conv2D(3,8,strides=1,padding='same')(input_)
x1=BatchNormalization()(x1)
x1=Activation('relu')(x1)
base_feat1=efn.EfficientNetB1(weights='imagenet',include_top=False)(x1)
x1=Reshape((base_feat1.shape[1]*base_feat1.shape[2],base_feat1.shape[3]))(base_feat1)

x2=Conv2D(3,12,strides=1,padding='same')(input_)
x2=BatchNormalization()(x2)
x2=Activation('relu')(x2)
base_feat2=efn.EfficientNetB2(weights='imagenet',include_top=False)(x2)
x2=Reshape((base_feat2.shape[1]*base_feat2.shape[2],base_feat2.shape[3]))(base_feat2)

x=Concatenate()([x0,x1,x2])
x=Bidirectional(LSTM(1024,return_sequences=True))(x)
x=Bidirectional(LSTM(512))(x)
x=Dense(1280,activation='relu')(x)
out=Dense(5,activation='sigmoid')(x)
model=Model(input_,out)

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])

In [ ]:
mc=ModelCheckpoint('classifier.h5',mode='min',monitor='val_loss')
rop=ReduceLROnPlateau(monitor='val_loss',mode='min',min_lr=0.0000001,verbose=1,patience=3)

In [ ]:
history=model.fit(train_gen,steps_per_epoch=train_gen.__len__(),epochs=20,
                  validation_data=val_gen,validation_steps=val_gen.__len__(),callbacks=[mc])

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'b',color='red', label='Training acc')
plt.plot(epochs, val_acc, 'b',color='blue', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', color='red', label='Training loss')
plt.plot(epochs, val_loss, 'b',color='blue', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()